In [1]:
# !~/3.9/bin/pip3 install instructor openai beautifulsoup4 lxml -U

In [2]:
# !~/3.9/bin/pip3 install git+https://github.com/mesolitica/malaya@5.1
# !~/3.9/bin/pip3 install torch --index-url https://download.pytorch.org/whl/cpu

In [3]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [5]:
instructor.patch()

In [6]:
engine = 'nous'

In [7]:
from typing import List

class Triplet(BaseModel):
    subject: str 
    predicate: str
    object: str
        
class KG(BaseModel):
    triplets: List[Triplet]

In [8]:
!mkdir paragraph-wikipedia

In [9]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine=engine,
            response_model=KG,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.dict()
    except:
        return None

In [ ]:
from tqdm import tqdm

with open('paragraph-wiki.jsonl') as fopen:
    for no, l in tqdm(enumerate(fopen)):
        filename = os.path.join('paragraph-wikipedia', f'{no}.json')
        if os.path.exists(filename):
            continue
        
        l = json.loads(l)
        bodies = []
        for b in l['paragraph']:
            bodies.append((b, predict(b)))
        l['paragraph_kg'] = bodies
        
        with open(filename, 'w') as fopen:
            json.dump(l, fopen)

0it [00:00, ?it/s]/tmp/ipykernel_1545335/4020766345.py:14: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  return response.dict()
25032it [74:23:47, 12.05s/it] 